<a href="https://colab.research.google.com/github/LamiaBensalem/YOLOv11-Exploration/blob/main/notebooks/YOLOv11_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [4]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [5]:
%pip install ultralytics supervision roboflow
import ultralytics
ultralytics.checks()

Ultralytics 8.3.54 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.0/107.7 GB disk)


In [6]:
from ultralytics import YOLO
from PIL import Image
import requests

# Load a pretrained YOLO11n model
model = YOLO("yolo11x.pt")

# Define path to directory containing images for inference
  # Base URL for raw images in the GitHub repository
base_url = "https://github.com/LamiaBensalem/YOLOv11-Exploration/blob/main/assests/"
  # List of image file names
file_names = [
    "street-in-hongkong.jpg",
    "Kora.jpg",
    "al-ain-zoo.jpg",
    "annaba-bike-club.jpg",
    "centennial-park.jpg",
    "dream-city.jpg",
    "first-case.jpg",
    "hill-house.jpg",
    "rene-magritte--the-art-of-conversation.jpg",
    "room.jpg"
]
  # List of image URLs
    # Construct the full URLs by appending file names to the base URL
image_urls = [f"{base_url}{name}?raw=true" for name in file_names]


# Open and process each image
images = []  # List to store opened images
for url in image_urls:
    try:
        response = requests.get(url, stream=True)
        img = Image.open(response.raw)  # Open image from the stream
        images.append(img)  # Add to list
        img.show()  # Display the image (optional)
    except Exception as e:
        print(f"Error fetching or opening image from {url}: {e}")


# Run inference on the source
source = image_urls
results = model.predict(source, stream = True, imgsz=320)  # list of Results objects
#for r in results:
    #im_array = r.plot()  # plot a BGR numpy array of predictions
    #im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    #r.show()  # show image

In [7]:
!pip install roboflow
!mkdir {HOME}/datasets
%cd {HOME}/datasets

from google.colab import userdata
from roboflow import Roboflow

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("microsoft").project("coco")
version = project.version(34)
dataset = version.download("yolov11")

print("\n Working Directory is",os.getcwd())

mkdir: cannot create directory ‘/content/datasets’: File exists
/content/datasets
loading Roboflow workspace...
loading Roboflow project...

 Working Directory is /content/datasets


In [ ]:
# Train the model
model = YOLO("yolo11s.pt")
data_location = os.getcwd()
results = model.train(data=f"{data_location}/COCO-Dataset-34/data.yaml", epochs=10, batch=16, imgsz=640, plots=True)


Ultralytics 8.3.54 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/datasets/COCO-Dataset-34/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

train: Scanning /content/datasets/COCO-Dataset-34/train/labels.cache... 135 images, 3 backgrounds, 0 corrupt: 100%|██████████| 135/135 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
val: Scanning /content/datasets/COCO-Dataset-34/valid/labels.cache... 55 images, 0 backgrounds, 0 corrupt: 100%|██████████| 55/55 [00:00<?, ?it/s]


Plotting labels to runs/detect/train5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000122, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train5
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G      1.146      5.998      1.175         33        640: 100%|██████████| 9/9 [05:54<00:00, 39.42s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:47<00:00, 23.59s/it]

                   all         55        397    0.00691       0.26     0.0186     0.0158



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/10         0G      1.138      5.515      1.174        114        640:  56%|█████▌    | 5/9 [03:24<02:42, 40.74s/it]

In [ ]:
!ls {HOME}/runs/detect/train/
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

IPyImage(filename=f'{HOME}/runs/detect/train/results.png', width=600)

IPyImage(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

In [ ]:
# Customize validation settings
validation_results = model.val(data=f"{data_location}/COCO-Dataset-34/data.yaml", imgsz=640, batch=16, conf=0.25, iou=0.6, device="0")

In [ ]:
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

In [ ]:
import shutil
shutil.rmtree('/content/datasets', ignore_errors=True)